# Doc2Vecの実行条件調査（part3）

- ラベルごとの教師データを最大３件とします（このためテストデータから学習セットの大半が失われます）


- 品詞を落とさないようにする


- パラメータを一部変更（DBOWを使用／feature数を調整）


accuracy=0.938 とややダウンしました。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

## (2) Doc2Vecの動作確認

### (2-1) コーパス生成

コーパス（単語が半角スペースで区切られた文字列）生成時、一部の品詞を落とすようにします。

（＝learning.core.nlang.Nlang クラスの仕様に従います）

In [2]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 9  # bot_id = 9はセプテーニ
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}

learning_parameter = LearningParameter(attr)

In [3]:
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])

2017/05/18 PM 01:51:38 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/18 PM 01:51:38 ['./fixtures/question_answers/toyotsu_human.csv']


In [4]:
import MeCab
import mojimoji

class Nlang_naive:
    @classmethod
    def split(self, text):
        tagger = MeCab.Tagger("-u learning/dict/custom.dic")
        tagger.parse('')  # node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(",")
            pos = features[0]
            if pos in ["BOS/EOS", "記号"]:
                node = node.next
                continue

            #print(features)
            lemma = node.feature.split(",")[6]

            if lemma == "*":
                lemma = node.surface  #.decode("utf-8")
                
            word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return " ".join(word_list)

    @classmethod
    def batch_split(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.split(text))
        return splited_texts

In [5]:
from learning.core.nlang import Nlang

_sentences = np.array(questions)
_separated_sentences = Nlang_naive.batch_split(_sentences)

#### ラベル毎の教師データ数を制限します。

In [6]:
answer_id_count = {}

selected_answer_ids = []
selected_questions = []
selected_separated_sentences = []

sample_limit_count = 3

for index, answer_id in enumerate(answer_ids):
    if answer_id not in answer_id_count.keys():
        answer_id_count[answer_id] = 0

    if answer_id_count[answer_id] < sample_limit_count:
        '''
            データをリストに格納
        '''
        selected_answer_ids.append('%04d_%02d' % (answer_id, answer_id_count[answer_id]))
        selected_questions.append(questions[index])
        selected_separated_sentences.append(_separated_sentences[index])
        answer_id_count[answer_id] += 1

### (2-2) コーパスにタグ付け

models.doc2vecの仕様に従います。

In [7]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

def doc_to_sentence(sentences, name):
    words = sentences.split(' ')
    return TaggedDocument(words=words, tags=[name])

def corpus_to_sentences(separated_sentences, answer_ids):
    for idx, (doc, name) in enumerate(zip(separated_sentences, answer_ids)):
        yield doc_to_sentence(doc, name)

### (2-3) 学習処理／モデルのシリアライズ

In [8]:
'''
    ユニークになったコーパス／ラベルから、
    学習セットを生成する
    
    教師データ件数＝519
'''
sentences = corpus_to_sentences(selected_separated_sentences, selected_answer_ids)
sentence_list = list(sentences)
len(sentence_list)

519

In [9]:
'''
    パラメータ変更点：
    　学習時にDBOWを使用する
    　featureの数をサンプル件数に近い値に設定
    　反復回数をfeature数の10倍に設定
'''
n_feature = round(len(sentence_list) + 49, -2) # 切り上げし、100の倍数に変更
n_iter = n_feature * 10
print('Doc2Vec: feature(size)=%d, iter=%d' % (n_feature, n_iter))

model = Doc2Vec(dm=0, size=n_feature, min_count=1, iter=n_iter)

Doc2Vec: feature(size)=600, iter=6000


In [10]:
'''
    ボキャブラリ生成／学習実行
    学習モデルは、ファイルに保存しておく
'''
model.build_vocab(sentence_list)
model.train(sentence_list)

model_path = 'prototype/better_algorithm/doc2vec.model'
model.save(model_path)

In [11]:
'''
    モデル内に保持されているベクトルの数を取得
    （教師データ数と同じであることを確認）
'''
len(model.docvecs)

519

In [12]:
model.docvecs

### (2-4) 予測処理

質問文は一部「揺れ」を与えたものとなっております

In [13]:
model_path = 'prototype/better_algorithm/doc2vec.model'

def predict(word, model_path):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang_naive.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(model_path)
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

In [14]:
'''
    マウスが破損（正解＝4458）
'''
predict('マウスが破損', model_path)

(['マウス', 'が', '破損'],
 [('4458_00', 0.8560611009597778),
  ('4564_01', 0.7289866805076599),
  ('4480_01', 0.7077137231826782),
  ('4564_02', 0.700944185256958),
  ('4480_00', 0.6949871182441711),
  ('4522_01', 0.668103814125061),
  ('4581_02', 0.665524959564209),
  ('4459_00', 0.665321409702301),
  ('4459_01', 0.6358392238616943),
  ('4530_01', 0.6333789229393005)])

In [15]:
'''
    無線を使用したい（正解＝4516）
'''
predict('無線を使用したい', model_path)

(['無線', 'を', '使用', 'する', 'たい'],
 [('4516_00', 0.8213694095611572),
  ('4595_02', 0.7438608407974243),
  ('4592_01', 0.7312545776367188),
  ('4429_00', 0.7244638204574585),
  ('4521_00', 0.7143065333366394),
  ('4491_02', 0.7135003805160522),
  ('4600_01', 0.7027838230133057),
  ('4577_01', 0.7015153765678406),
  ('4592_00', 0.6931694149971008),
  ('4587_02', 0.6652255654335022)])

In [16]:
'''
    情報システムのアドレス（正解＝7040）
'''
predict('情報システムのアドレス', model_path)

(['情報', 'システム', 'の', 'アドレス'],
 [('7040_01', 0.8801047801971436),
  ('7040_02', 0.7885791063308716),
  ('4547_00', 0.6965842843055725),
  ('4514_01', 0.6963896155357361),
  ('7040_00', 0.6550189256668091),
  ('7065_00', 0.6480227112770081),
  ('4455_02', 0.6279381513595581),
  ('7065_01', 0.6156381964683533),
  ('7065_02', 0.6130967140197754),
  ('4618_02', 0.6058664321899414)])

In [17]:
'''
    Office2010を使用したいのですが（正解＝4625）
'''
predict('Office2010を使用したいのですが', model_path)

(['Ｏｆｆｉｃｅ', '２０１０', 'を', '使用', 'する', 'たい', 'の', 'です', 'が'],
 [('4625_00', 0.7719082832336426),
  ('4595_00', 0.7016371488571167),
  ('4516_00', 0.679406464099884),
  ('4429_00', 0.6326416730880737),
  ('4523_00', 0.627448558807373),
  ('4533_00', 0.625619113445282),
  ('4625_01', 0.6239690780639648),
  ('4600_01', 0.6117346286773682),
  ('4521_00', 0.6094182133674622),
  ('4592_00', 0.6045582294464111)])

In [18]:
'''
    携帯からサイボウズを使いたいのですが、どうしたら出来ますか？
    （正解＝4504だが、学習時の教師データから抜け落ちたもの）
'''
predict('携帯からサイボウズを使いたいのですが、どうしたら出来ますか？', model_path)

(['携帯',
  'から',
  'サイボウズ',
  'を',
  '使う',
  'たい',
  'の',
  'です',
  'が',
  'どう',
  'する',
  'た',
  '出来る',
  'ます',
  'か'],
 [('4541_02', 0.6198078393936157),
  ('4533_02', 0.6113443374633789),
  ('7042_01', 0.5967990756034851),
  ('4541_00', 0.594677209854126),
  ('4533_01', 0.5906194448471069),
  ('4518_02', 0.5768016576766968),
  ('4501_00', 0.5588986277580261),
  ('4533_00', 0.558854877948761),
  ('7042_00', 0.5532267689704895),
  ('7056_00', 0.5493056774139404)])

## (3) accuracy 測定

ここからが本題です

In [19]:
model_path = 'prototype/better_algorithm/doc2vec.model'

len(selected_separated_sentences)

519

In [20]:
def predict_similarity(separated_sentence, model_path):
    corpus = separated_sentence.split()
    loaded_model = models.Doc2Vec.load(model_path)
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    answer_id, similarity = ret[0]
    return corpus, answer_id, similarity

In [21]:
def get_prediction_statistics(separated_sentences, answer_ids, model_path):
    '''
        学習セットの質問文をそのまま予測処理にかけて、
        回答を予測
    '''
    statistics = []
    for i, _ in enumerate(separated_sentences):
        sentence = separated_sentences[i]
        preferred_answer_id = answer_ids[i]
        corpus, answer_id, similarity = predict_similarity(separated_sentences[i], model_path)
        corpus_len = len(corpus)
        statistics.append((i, corpus_len, preferred_answer_id, answer_id, similarity))

    return statistics

In [22]:
prediction_statistics = get_prediction_statistics(selected_separated_sentences, selected_answer_ids, model_path)

In [23]:
ncorrect_by_corpus_len = {}
nsample_by_corpus_len = {}

ncorrect = 0
nsample = 0

'''
    予測結果を、質問文の単語数毎／回答ID毎に統計する
'''
for statistics in prediction_statistics:
    i, corpus_len, preferred_answer_id, answer_id, similarity = statistics
    
    '''
        質問文の単語数ごとに統計を取る
    '''
    if corpus_len not in nsample_by_corpus_len.keys():
        ncorrect_by_corpus_len[corpus_len] = 0
        nsample_by_corpus_len[corpus_len] = 0
    nsample_by_corpus_len[corpus_len] += 1
    
    '''
        正解かどうか検査
        （NNNN_nn 形式ラベルの上４桁が一致していれば正解とします）
    '''
    nsample += 1
    if preferred_answer_id[0:4] == answer_id[0:4]:
        ncorrect += 1
        ncorrect_by_corpus_len[corpus_len] += 1

In [24]:
'''
    質問文の単語数ごとの統計情報を編集
'''
info_by_corpus_len = []
for k, v in ncorrect_by_corpus_len.items():
    info_by_corpus_len.append((
        k, 
        ncorrect_by_corpus_len[k]/nsample_by_corpus_len[k], 
        ncorrect_by_corpus_len[k], 
        nsample_by_corpus_len[k]
    ))

In [25]:
'''
    全体の正解率
'''
print("accuracy=%0.3f (%d/%d)" % (
    ncorrect/nsample, ncorrect, nsample))

accuracy=0.938 (487/519)


In [26]:
'''
    質問文の単語数ごとの正解率をリスト
'''
for info in info_by_corpus_len:
    print("word_count=%2d: accuracy=%0.3f (%d/%d)" % (
        info[0], info[1], info[2], info[3]
    ))

word_count= 1: accuracy=1.000 (1/1)
word_count= 2: accuracy=0.921 (35/38)
word_count= 3: accuracy=0.969 (94/97)
word_count= 4: accuracy=0.955 (106/111)
word_count= 5: accuracy=0.873 (62/71)
word_count= 6: accuracy=0.818 (36/44)
word_count= 7: accuracy=0.917 (33/36)
word_count= 8: accuracy=1.000 (33/33)
word_count= 9: accuracy=0.938 (15/16)
word_count=10: accuracy=1.000 (8/8)
word_count=11: accuracy=1.000 (15/15)
word_count=12: accuracy=1.000 (12/12)
word_count=13: accuracy=1.000 (12/12)
word_count=14: accuracy=1.000 (6/6)
word_count=15: accuracy=1.000 (6/6)
word_count=16: accuracy=1.000 (4/4)
word_count=17: accuracy=1.000 (2/2)
word_count=18: accuracy=1.000 (1/1)
word_count=19: accuracy=1.000 (1/1)
word_count=20: accuracy=1.000 (1/1)
word_count=22: accuracy=1.000 (1/1)
word_count=29: accuracy=1.000 (1/1)
word_count=30: accuracy=1.000 (1/1)
word_count=31: accuracy=1.000 (1/1)
